# CONV LSTM Univariate

A type of LSTM related to the CNN-LSTM is the ConvLSTM, where the convolutional reading
of input is built directly into each LSTM unit. The ConvLSTM was developed for reading
two-dimensional spatial-temporal data, but can be adapted for use with univariate time series
forecasting.

In [1]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import ConvLSTM2D

Using TensorFlow backend.


#### split a univariate sequence into samples

In [2]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

## 1. Data Preparation

#### define input sequence

In [3]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

#### choose a number of time steps

In [4]:
n_steps = 4

#### split into samples

In [5]:
X, y = split_sequence(raw_seq, n_steps)

In [6]:
X

array([[10, 20, 30, 40],
       [20, 30, 40, 50],
       [30, 40, 50, 60],
       [40, 50, 60, 70],
       [50, 60, 70, 80]])

#### reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]

In [7]:
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))

In [8]:
X

array([[[[[10],
          [20]]],


        [[[30],
          [40]]]],



       [[[[20],
          [30]]],


        [[[40],
          [50]]]],



       [[[[30],
          [40]]],


        [[[50],
          [60]]]],



       [[[[40],
          [50]]],


        [[[60],
          [70]]]],



       [[[[50],
          [60]]],


        [[[70],
          [80]]]]])

## 2. CONV LSTM

#### define model

In [9]:
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

#### fit model

In [10]:
model.fit(X, y, epochs=500, verbose=0)

#### demonstrate prediction

In [11]:
x_input = array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, 1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[103.74684]]
